In [1]:
import andi
import numpy as np
import csv as csv
import numpy as np

from sklearn.metrics import mean_absolute_error
import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
from data_split import data_split

Using TensorFlow backend.


Importing the data

In [2]:
import csv
trajs_from_files = csv.reader(open('challenge_for_scoring/task3.txt','r'), delimiter=';', 
                                        lineterminator='\n',quoting=csv.QUOTE_NONNUMERIC)
validation = [[],[],[]]
for trajs in enumerate(trajs_from_files):
    validation[int(trajs[1][0])-1].append(trajs[1][1:])

Importing the networks

In [3]:

model_switch_a_t_new = load_model('task3_new.h5')

model_switch_a_t_diff = load_model('diff_task3.h5')




model_classi_first = load_model('task3_classi_first10249.h5')

#model_classi_sec = load_model('task3_classi_second.h5')

#using new net, NB this requires diff and no time stamp

model_classi_sec = load_model('taks_3_classify_second_new.h5')

Personalized arctan to convert predictions into switching times stats

In [4]:
def my_atan(x1,x2):
    y=np.arctan2(x1,x2)
    b=y<0
    c=b.astype(int)*(2*np.pi)
    d=y+c 
    return    d;

In [5]:
#for 1d
i=200
validation1d=validation[0]
test_tim_step=np.arange(200)
show_time_coll=np.tile(test_tim_step,(len(validation1d),1))
data_show,label_show,traj_show,times_show=data_split(np.asarray(validation1d),
                                                     show_time_coll,
                                                         labels=np.ones(len(validation1d)),
                                                         start_row=0,num_row=len(validation1d),
                                                         traj_len=np.asarray(validation1d).shape[1],n_in=0,n_samples=1,
                                                         p_p=1,hmin=0,hmax=2,limith=False,normalization=True)
                                                     
# pred_a_t=model_switch_a_t.predict(data_show)
# pred_a_t_int=model_switch_a_t_int(data_show)
pred_a_t_new=model_switch_a_t_new(data_show)


#model classification

pred_m1_first=np.argmax(model_classi_first.predict(data_show),axis=1)

#pred_m2_sec=np.argmax(model_classi_sec.predict(data_show),axis=1)

In [6]:
#Estimating the exponents and switching point using diff
i=200

data_show_diff,label_show,traj_show,times_show=data_split(np.diff(np.asarray(validation1d),axis=1),
                                                     show_time_coll,
                                                         labels=np.ones(len(validation1d)),
                                                         start_row=0,num_row=len(validation1d),
                                                         traj_len=i-1,n_in=0,n_samples=1,
                                                         p_p=1,hmin=0,hmax=2,limith=False,normalization=True)
pred_a_t_diff=model_switch_a_t_diff.predict(data_show_diff)
pred_m2_sec=np.argmax(model_classi_sec.predict(traj_show.reshape((-1,199,1))),axis=1)

In [ ]:
#Estimating the exponents and switching point using the reversed trajectory
# i=200

# data_show_rev,label_show,traj_show,times_show=data_split(np.fliplr(np.asarray(validation)),
#                                                      show_time_coll,
#                                                          labels=np.ones(len(validation)),
#                                                          start_row=0,num_row=len(validation),
#                                                          traj_len=i,n_in=0,n_samples=1,
#                                                          p_p=1,hmin=0,hmax=2,limith=False,normalization=True)
# pred_a_t_rev=model_switch_a_t_rev.predict(data_show_rev)

In [7]:
#combining predictions for the switching time
# pr_t = my_atan(pred_a_t[:,2],pred_a_t[:,3])*200/(2*np.pi)

# pr_t_rev = my_atan(pred_a_t_rev[:,2],pred_a_t_rev[:,3])*200/(2*np.pi)

# pr_t_int = my_atan(pred_a_t_int[:,2],pred_a_t_int[:,3])*200/(2*np.pi)

pr_t_new = my_atan(pred_a_t_new[:,2],pred_a_t_new[:,3])*200/(2*np.pi)

pr_t_diff = my_atan(pred_a_t_diff[:,2],pred_a_t_diff[:,3])*200/(2*np.pi)




pr_t_comb = (pr_t_new+pr_t_diff)/2

pred_m1s=pred_m1_first
pred_m2s=pred_m2_sec

#assembling the total prediction array now giving (t,model 1,a1,model2,a2)
predictions=np.zeros((len(validation1d),5))
predictions[:,0]=pr_t_comb
predictions[:,1]=pred_m1s
predictions[:,2]=(pred_a_t_new[:,0]+pred_a_t_diff[:,0])/2
predictions[:,3]=pred_m2s
predictions[:,4]=(pred_a_t_new[:,1]+pred_a_t_diff[:,1])/2


In [8]:
np.save('task3_predictions1d_new.npy',predictions)

In [9]:
predictions1d=np.copy(predictions)

# 2d data

In [ ]:
#for 2d
i=200
for dim in [2]:
    pred_hd_a_t=np.zeros((len(validation[1]),3))
    
    pred_hd_m1_first=np.zeros((len(validation[1]),5))
    pred_hd_m2_sec=np.zeros((len(validation[1]),5))
    valid_ch_dim=np.asarray(validation[dim-1])    #validation in the chosen dimension
    for jj in range(dim):
        x=valid_ch_dim[:,i*jj:i*(jj+1)]
        test_tim_step=np.arange(i)
        show_time_coll=np.tile(test_tim_step,(len(x),1))
        data_show,label_show,traj_show,times_show=data_split(x,
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1],n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

        # pred_a_t=model_switch_a_t.predict(data_show)
        # pred_a_t_int=model_switch_a_t_int(data_show)
        pred_a_t_new=model_switch_a_t_new(data_show)
        pr_t_new = my_atan(pred_a_t_new[:,2],pred_a_t_new[:,3])*200/(2*np.pi)
        pred_hd_a_t[:,:2]+=pred_a_t_new[:,:2]/dim
        pred_hd_a_t[:,2]+=pr_t_new/dim
        #model classification

        pred_m1_first=model_classi_first.predict(data_show)
        pred_hd_m1_first+=pred_m1_first/dim
        
        data_show_diff,label_show,traj_show_diff,times_show=data_split(np.diff(x,axis=1),
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1]-1,n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

        
        pred_m2_sec=model_classi_sec.predict(traj_show_diff.reshape((-1,199,1)))
        pred_hd_m2_sec+=pred_m2_sec/dim
        
    #combining the different dimensions
    

In [ ]:
predictions2d[:,0].shape

In [ ]:
predictions2d=np.zeros((len(validation[0]),5))
predictions2d[:,0]=pred_hd_a_t[:,2]  # t switch
predictions2d[:,1]=np.argmax(pred_hd_m1_first,axis=1)
predictions2d[:,2]=pred_hd_a_t[:,0]  # a1
predictions2d[:,3]=np.argmax(pred_hd_m2_sec,axis=1)
predictions2d[:,4]=pred_hd_a_t[:,1]  #a2

In [ ]:
np.save('task3_predictions2d_new.npy',predictions2d)

In [ ]:
#for 3d
i=200
for dim in [3]:
    pred_hd_a_t=np.zeros((len(validation[0]),3))
    
    pred_hd_m1_first=np.zeros((len(validation[0]),5))
    pred_hd_m2_sec=np.zeros((len(validation[0]),5))
    valid_ch_dim=np.asarray(validation[dim-1])    #validation in the chosen dimension
    for jj in range(dim):
        x=valid_ch_dim[:,i*jj:i*(jj+1)]
        test_tim_step=np.arange(i)
        show_time_coll=np.tile(test_tim_step,(len(x),1))
        data_show,label_show,traj_show,times_show=data_split(x,
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1],n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

        # pred_a_t=model_switch_a_t.predict(data_show)
        # pred_a_t_int=model_switch_a_t_int(data_show)
        pred_a_t_new=model_switch_a_t_new(data_show)
        pr_t_new = my_atan(pred_a_t_new[:,2],pred_a_t_new[:,3])*200/(2*np.pi)
        pred_hd_a_t[:,:2]+=pred_a_t_new[:,:2]/dim
        pred_hd_a_t[:,2]+=pr_t_new/dim
        #model classification

        pred_m1_first=model_classi_first.predict(data_show)
        pred_hd_m1_first+=pred_m1_first/dim
        
        data_show_diff,label_show,traj_show_diff,times_show=data_split(np.diff(x,axis=1),
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1]-1,n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

                
        
        
        pred_m2_sec=model_classi_sec.predict(traj_show_diff.reshape((-1,199,1)))
        pred_hd_m2_sec+=pred_m2_sec/dim
        
predictions3d=np.zeros((len(validation[0]),5))
predictions3d[:,0]=pred_hd_a_t[:,2]  # t switch
predictions3d[:,1]=np.argmax(pred_hd_m1_first,axis=1)
predictions3d[:,2]=pred_hd_a_t[:,0]  # a1
predictions3d[:,3]=np.argmax(pred_hd_m2_sec,axis=1)
predictions3d[:,4]=pred_hd_a_t[:,1]  #a2    

In [ ]:
np.save('task3_predictions3d_new.npy',predictions3d)

In [28]:
pred_m1_first[:2]

array([[0.3000441 , 0.07568342, 0.26319143, 0.04060716, 0.32047388],
       [0.06691413, 0.6945819 , 0.20749769, 0.00179599, 0.02921026]],
      dtype=float32)

In [29]:
pred_hd_m1_first[:2]

array([[0.23652896, 0.04030416, 0.24889491, 0.08138584, 0.39288618],
       [0.09313251, 0.56445937, 0.26713513, 0.00262254, 0.07265043]])

In [24]:
pred_to_txt2d = 2*np.ones((len(predictions2d), 6))
for i in range(len(predictions2d)):
    for j in range(5):
        
        pred_to_txt2d[i, j+1] = predictions2d[i][j]
pred_to_txt = np.concatenate((pred_to_txt1d,pred_to_txt2d))
np.savetxt('task3_tem2d.txt', pred_to_txt.astype(float), fmt = '%1.5f', delimiter = ';')

In [41]:
#pred_to_txt1d=np.genfromtxt('task3.txt', delimiter = ';')

In [25]:
pred_to_txt3d = 3*np.ones((len(predictions3d), 6))
for i in range(len(predictions3d)):
    for j in range(5):
        
        pred_to_txt3d[i, j+1] = predictions3d[i][j]
pred_to_txt_fin = np.concatenate((pred_to_txt,pred_to_txt3d))
np.savetxt('task3_tem3d.txt', pred_to_txt_fin.astype(float), fmt = '%1.5f', delimiter = ';')

In [26]:
predictions3d.shape

(10000, 5)

In [27]:
predictions2d.shape

(10000, 5)

In [28]:
predictions1d.shape

(10000, 5)